## Iteration 5

In the fifth iteration, it was decided to refactor the code to use idiomatic LangChain patterns while keeping it simple and linear.

Key changes:
- Added Pydantic models for typed outputs (MathContent, JudgmentResult)
- Used FewShotChatMessagePromptTemplate for few-shot examples
- Used ChatPromptTemplate.from_messages() instead of manual message lists
- Used PydanticOutputParser for automatic JSON parsing
- Created chains with LCEL: prompt | model | parser

In [21]:
import json
from typing import Literal
import jsonlines
import pandas as pd
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

In [22]:
pd.set_option('display.max_colwidth', None)

In [23]:
load_dotenv()

True

In [24]:
class MathContent(BaseModel):
    type: Literal["Задача", "Тест"]
    description: str
    solution: str
    answer: str

class JudgmentResult(BaseModel):
    has_numbered_solution: bool
    has_valid_answer_format: bool
    has_valid_description_format: bool

In [25]:
with jsonlines.open('dataset.jsonl') as reader:
    dataset = list(reader)

dataset[0]

{'topic': 'Арифметичні дії',
 'type': 'task',
 'description': 'Обчисліть значення виразу: $100 - (15 + 5) \\times 4$.',
 'solution': '1. Виконайте дію в дужках: $15 + 5 = 20$. 2. Виконайте множення отриманого результату на 4: $20 \\times 4 = 80$. 3. Виконайте віднімання від 100: $100 - 80 = 20$.',
 'answer': '20'}

In [26]:
generator_model = ChatOpenAI(model="gpt-4o", temperature=0.7)
judge_model = ChatOpenAI(model="gpt-4o", temperature=0.1)

In [27]:
class MathTutor:
    def __init__(self, model):
        self.model = model
        self.parser = PydanticOutputParser(pydantic_object=MathContent)
        
        self.system_prompt = """Ти - вчитель математики. Проаналізуй запит користувача та надай відповідь.

Якщо користувач вказав тип контенту (Задача/Тест) - використай його. Якщо ні - обери відповідний тип самостійно.

Типи контенту:
1. Задача: 
   - description: умова задачі з одним питанням
   - solution: нумерований список кроків розв'язання
   - answer: число або вираз (без додатковий позначень)

2. Тест:
   - description: умова задачі з чотирма варіантами відповіді (A, B, C, D)
   - solution: нумерований список кроків для знаходження відповіді
   - answer: одна латинська літера (A, B, C або D)

Відповідь надай у JSON форматі з чотирма полями: type (Задача або Тест), description, solution, answer.
Не обгортай JSON у додаткові спецсимволи."""

        self.examples = [
            {"input": "Тема: Площа круга, Тип: Задача", "output": '{"type": "Задача", "description": "Обчисліть площу круга з радіусом $r = 4$. Прийміть $\\\\pi \\\\approx 3.14$.", "solution": "1. Згадайте формулу площі круга: $S = \\\\pi r^2$. 2. Підставте значення радіуса та числа $\\\\pi$: $S \\\\approx 3.14 \\\\cdot 4^2$. 3. Піднесіть радіус до квадрата: $4^2 = 16$. 4. Виконайте множення: $3.14 \\\\cdot 16 = 50.24$.", "answer": "50.24"}'},
            {"input": "Знайдіть корінь рівняння $2x - 5 = 7$.", "output": '{"type": "Задача", "description": "Знайдіть корінь рівняння $2x - 5 = 7$.", "solution": "1. Перенесіть вільний член (-5) у праву частину рівняння, змінивши знак на протилежний: $2x = 7 + 5$. 2. Обчисліть суму в правій частині: $2x = 12$. 3. Поділіть обидві частини рівняння на коефіцієнт при змінній (на 2): $x = 12 / 2$. 4. Отримайте відповідь: $x = 6$.", "answer": "6"}'},
            {"input": "Чому дорівнює $\\sqrt{144}$? \\n A) 10 \\n B) 11 \\n C) 12 \\n D) 14", "output": '{"type": "Тест", "description": "Чому дорівнює $\\\\sqrt{144}$? \\\\n A) 10 \\\\n B) 11 \\\\n C) 12 \\\\n D) 14", "solution": "1. Визначте, що потрібно знайти число, яке при множенні саме на себе дає 144. 2. Перевірте варіанти відповідей піднесенням до квадрату. 3. $10^2=100$, $11^2=121$, $12^2=144$. 4. Оскільки $12^2=144$, оберіть варіант C.", "answer": "C"}'},
        ]
        
        example_prompt = ChatPromptTemplate.from_messages([
            ("human", "{input}"),
            ("ai", "{output}")
        ])
        
        few_shot_prompt = FewShotChatMessagePromptTemplate(
            examples=self.examples,
            example_prompt=example_prompt
        )
        
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", self.system_prompt),
            few_shot_prompt,
            ("human", "{user_input}")
        ])
        
        self.chain = self.prompt | self.model | self.parser

    def invoke(self, user_input: str) -> MathContent:
        return self.chain.invoke({"user_input": user_input})

In [28]:
class AcceptanceJudge:
    def __init__(self, model):
        self.model = model
        self.parser = PydanticOutputParser(pydantic_object=JudgmentResult)
        
        self.system_prompt = """Ти - експерт з оцінювання математичного контенту. Оціни наданий контент за критеріями.

Критерії:
1. has_numbered_solution: чи містить поле solution нумерований список кроків (1. ... 2. ... тощо)
2. has_valid_answer_format: 
   - для типу "Задача": відповідь має бути числом або виразом, або їх комбінацією
   - для типу "Тест": відповідь має бути однією латинською літерою (A, B, C або D)
3. has_valid_description_format:
   - для типу "Задача": опис має містити умову задачі
   - для типу "Тест": опис має містити чотири варіанти відповіді (A, B, C, D)

Відповідь надай у JSON форматі з трьома булевими полями: has_numbered_solution, has_valid_answer_format, has_valid_description_format.
Не обгортай JSON у додаткові спецсимволи."""

        self.examples = [
            {"input": 'Тип контенту: Задача\n\nКонтент:\n{"description": "Обчисліть площу круга з радіусом $r = 4$. Прийміть $\\\\pi \\\\approx 3.14$.", "solution": "1. Згадайте формулу площі круга: $S = \\\\pi r^2$. 2. Підставте значення радіуса та числа $\\\\pi$: $S \\\\approx 3.14 \\\\cdot 4^2$. 3. Піднесіть радіус до квадрата: $4^2 = 16$. 4. Виконайте множення: $3.14 \\\\cdot 16 = 50.24$.", "answer": "50.24"}', "output": '{"has_numbered_solution": true, "has_valid_answer_format": true, "has_valid_description_format": true}'},
            {"input": 'Тип контенту: Тест\n\nКонтент:\n{"description": "Знайдіть корінь рівняння $2x - 5 = 7$. \\n A) 1 \\n B) 6 \\n C) 12 \\n D) 2", "solution": "1. Перенесіть вільний член (-5) у праву частину рівняння, змінивши знак на протилежний: $2x = 7 + 5$. 2. Обчисліть суму в правій частині: $2x = 12$. 3. Поділіть обидві частини рівняння на коефіцієнт при змінній (на 2): $x = 12 / 2$. 4. Отримайте $x = 6$ і оберіть відповідний варіант B.", "answer": "B"}', "output": '{"has_numbered_solution": true, "has_valid_answer_format": true, "has_valid_description_format": true}'},
        ]
        
        example_prompt = ChatPromptTemplate.from_messages([
            ("human", "{input}"),
            ("ai", "{output}")
        ])
        
        few_shot_prompt = FewShotChatMessagePromptTemplate(
            examples=self.examples,
            example_prompt=example_prompt
        )
        
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", self.system_prompt),
            few_shot_prompt,
            ("human", "Тип контенту: {content_type}\n\nКонтент:\n{content}")
        ])
        
        self.chain = self.prompt | self.model | self.parser

    def evaluate(self, content: MathContent) -> JudgmentResult:
        content_json = json.dumps(content.model_dump(), ensure_ascii=False)
        return self.chain.invoke({"content_type": content.type, "content": content_json})

In [29]:
tutor = MathTutor(generator_model)
judge = AcceptanceJudge(judge_model)

def evaluate_input(user_input: str):
    try:
        content = tutor.invoke(user_input)
        tutor_success = True
    except Exception:
        tutor_success = False
        content = None
    
    if content:
        try:
            judgment = judge.evaluate(content)
            judge_success = True
        except Exception:
            judge_success = False
            judgment = None
    else:
        judge_success = None
        judgment = None
    
    display(pd.DataFrame([
        {'field': 'user_input', 'value': user_input},
        {'field': 'tutor_success', 'value': tutor_success},
        {'field': 'tutor_type', 'value': content.type if content else ''},
        {'field': 'tutor_description', 'value': content.description if content else ''},
        {'field': 'tutor_solution', 'value': content.solution if content else ''},
        {'field': 'tutor_answer', 'value': content.answer if content else ''},
        {'field': 'judge_success', 'value': judge_success},
        {'field': 'judge_has_numbered_solution', 'value': judgment.has_numbered_solution if judgment else None},
        {'field': 'judge_has_valid_answer_format', 'value': judgment.has_valid_answer_format if judgment else None},
        {'field': 'judge_has_valid_description_format', 'value': judgment.has_valid_description_format if judgment else None},
    ]))

    return {
        'user_input': user_input[:50] + '...' if len(user_input) > 50 else user_input,
        'tutor_success': tutor_success,
        'tutor_type': content.type if content else '',
        'judge_success': judge_success,
        'judge_has_numbered_solution': judgment.has_numbered_solution if judgment else None,
        'judge_has_valid_answer_format': judgment.has_valid_answer_format if judgment else None,
        'judge_has_valid_description_format': judgment.has_valid_description_format if judgment else None,
    }

results = []

for item in dataset[:5]:
    results.append(evaluate_input(item['topic']))

for item in dataset[:5]:
    results.append(evaluate_input(item['description']))

results = pd.DataFrame(results)

,field,value
0,user_input,Арифметичні дії
1,tutor_success,True
2,tutor_type,Задача
3,tutor_description,Обчисліть значення виразу $7 + 3 \times 2 - 8 / 4$.
4,tutor_solution,1. Виконайте множення: $3 \times 2 = 6$. 2. Виконайте ділення: $8 / 4 = 2$. 3. Обчисліть суму: $7 + 6 = 13$. 4. Від отриманої суми відніміть результат ділення: $13 - 2 = 11$.
5,tutor_answer,11
6,judge_success,True
7,judge_has_numbered_solution,True
8,judge_has_valid_answer_format,True
9,judge_has_valid_description_format,True


,field,value
0,user_input,Геометрія
1,tutor_success,True
2,tutor_type,Задача
3,tutor_description,"У прямокутному трикутнику один з катетів дорівнює 6 см, а гіпотенуза дорівнює 10 см. Знайдіть довжину другого катета."
4,tutor_solution,"1. Використайте теорему Піфагора: $c^2 = a^2 + b^2$, де $c$ - гіпотенуза, $a$ і $b$ - катети. 2. Підставте відомі значення: $10^2 = 6^2 + b^2$. 3. Обчисліть: $100 = 36 + b^2$. 4. Відніміть 36 з обох сторін: $b^2 = 64$. 5. Знайдіть $b$ шляхом вилучення квадратного кореня: $b = \sqrt{64} = 8$ см."
5,tutor_answer,8
6,judge_success,True
7,judge_has_numbered_solution,True
8,judge_has_valid_answer_format,True
9,judge_has_valid_description_format,True


,field,value
0,user_input,Спрощення виразів
1,tutor_success,True
2,tutor_type,Задача
3,tutor_description,Спростіть вираз $3(x + 4) - 2(x - 1) + 5$.
4,tutor_solution,"1. Розкрийте дужки у виразі: $3(x + 4) - 2(x - 1) + 5 = 3x + 12 - 2x + 2 + 5$. 2. Поєднайте подібні члени: $3x - 2x = x$, $12 + 2 + 5 = 19$. 3. Отримайте спрощений вираз: $x + 19$."
5,tutor_answer,x + 19
6,judge_success,True
7,judge_has_numbered_solution,True
8,judge_has_valid_answer_format,True
9,judge_has_valid_description_format,True


,field,value
0,user_input,Нерівності
1,tutor_success,True
2,tutor_type,Задача
3,tutor_description,Розв'яжіть нерівність $3x + 7 < 16$.
4,tutor_solution,"1. Перенесіть вільний член (7) у праву частину нерівності, змінивши знак на протилежний: $3x < 16 - 7$. 2. Обчисліть різницю в правій частині: $3x < 9$. 3. Поділіть обидві частини нерівності на коефіцієнт при змінній (на 3): $x < 9 / 3$. 4. Отримайте відповідь: $x < 3$."
5,tutor_answer,x < 3
6,judge_success,True
7,judge_has_numbered_solution,True
8,judge_has_valid_answer_format,True
9,judge_has_valid_description_format,True


,field,value
0,user_input,Системи рівнянь
1,tutor_success,True
2,tutor_type,Задача
3,tutor_description,Розв'яжіть систему рівнянь: \( \begin{cases} 2x + 3y = 12 \ 4x - y = 5 \end{cases} \)
4,tutor_solution,1. Виразіть одну з змінних з одного з рівнянь. Візьмемо друге рівняння: $4x - y = 5$. Виразіть $y$: $y = 4x - 5$. 2. Підставте вираз для $y$ в перше рівняння: $2x + 3(4x - 5) = 12$. 3. Розкрийте дужки: $2x + 12x - 15 = 12$. 4. Спростіть рівняння: $14x - 15 = 12$. 5. Перенесіть $-15$ в праву частину: $14x = 27$. 6. Поділіть на 14: $x = \frac{27}{14}$. 7. Підставте значення $x$ в вираз для $y$: $y = 4 \cdot \frac{27}{14} - 5$. 8. Обчисліть $y$: $y = \frac{108}{14} - 5 = \frac{108}{14} - \frac{70}{14} = \frac{38}{14} = \frac{19}{7}$.
5,tutor_answer,"x = \frac{27}{14}, y = \frac{19}{7}"
6,judge_success,True
7,judge_has_numbered_solution,True
8,judge_has_valid_answer_format,True
9,judge_has_valid_description_format,True


,field,value
0,user_input,Обчисліть значення виразу: $100 - (15 + 5) \times 4$.
1,tutor_success,True
2,tutor_type,Задача
3,tutor_description,Обчисліть значення виразу: $100 - (15 + 5) \times 4$.
4,tutor_solution,1. Обчисліть значення у дужках: $15 + 5 = 20$. 2. Помножте результат на 4: $20 \times 4 = 80$. 3. Відніміть отримане значення з 100: $100 - 80$. 4. Отримайте результат: $20$.
5,tutor_answer,20
6,judge_success,True
7,judge_has_numbered_solution,True
8,judge_has_valid_answer_format,True
9,judge_has_valid_description_format,True


,field,value
0,user_input,Чому дорівнює сума внутрішніх кутів довільного трикутника? \n A) $90^\circ$ \n B) $180^\circ$ \n C) $270^\circ$ \n D) $360^\circ$
1,tutor_success,True
2,tutor_type,Тест
3,tutor_description,Чому дорівнює сума внутрішніх кутів довільного трикутника? \n A) $90^\circ$ \n B) $180^\circ$ \n C) $270^\circ$ \n D) $360^\circ$
4,tutor_solution,"1. Згадайте властивість про те, що сума внутрішніх кутів трикутника завжди дорівнює $180^\circ$. 2. Перевірте варіанти відповідей. 3. Оберіть варіант, що відповідає цій властивості, а саме варіант B."
5,tutor_answer,B
6,judge_success,True
7,judge_has_numbered_solution,True
8,judge_has_valid_answer_format,True
9,judge_has_valid_description_format,True


,field,value
0,user_input,"Спростіть вираз, використовуючи формулу різниці квадратів: $(x - 3)(x + 3)$."
1,tutor_success,True
2,tutor_type,Задача
3,tutor_description,"Спростіть вираз, використовуючи формулу різниці квадратів: $(x - 3)(x + 3)$."
4,tutor_solution,"1. Пригадайте формулу різниці квадратів: $(a - b)(a + b) = a^2 - b^2$. 2. Застосуйте формулу до виразу $(x - 3)(x + 3)$, де $a = x$ і $b = 3$. 3. Обчисліть за формулою: $x^2 - 3^2$. 4. Обчисліть $3^2 = 9$. 5. Отримайте спрощений вираз: $x^2 - 9$."
5,tutor_answer,x^2 - 9
6,judge_success,True
7,judge_has_numbered_solution,True
8,judge_has_valid_answer_format,True
9,judge_has_valid_description_format,True


,field,value
0,user_input,Який проміжок є розв'язком нерівності $x > 5$? \n A) $(-\infty; 5)$ \n B) $[5; +\infty)$ \n C) $(5; +\infty)$ \n D) $(-\infty; 5]$
1,tutor_success,True
2,tutor_type,Тест
3,tutor_description,Який проміжок є розв'язком нерівності $x > 5$? \n A) $(-\infty; 5)$ \n B) $[5; +\infty)$ \n C) $(5; +\infty)$ \n D) $(-\infty; 5]$
4,tutor_solution,"1. Згадайте, що нерівність $x > 5$ означає, що $x$ більше за 5, але не дорівнює 5. 2. Розв'язок має включати всі числа, більші за 5, але не включати саме число 5. 3. Проміжок, що задовольняє цю умову, - це $(5; +\infty)$. 4. Оберіть варіант C."
5,tutor_answer,C
6,judge_success,True
7,judge_has_numbered_solution,True
8,judge_has_valid_answer_format,True
9,judge_has_valid_description_format,True


,field,value
0,user_input,Розв'яжіть систему рівнянь: $\begin{cases} x + y = 10 \\ x - y = 4 \end{cases}$
1,tutor_success,True
2,tutor_type,Задача
3,tutor_description,Розв'яжіть систему рівнянь: $\begin{cases} x + y = 10 \\ x - y = 4 \end{cases}$
4,tutor_solution,"1. Складіть обидва рівняння системи, щоб позбутися $y$: $(x + y) + (x - y) = 10 + 4$. 2. Отримайте рівняння $2x = 14$. 3. Поділіть обидві частини рівняння на 2: $x = 7$. 4. Підставте значення $x = 7$ у перше рівняння системи: $7 + y = 10$. 5. Відніміть 7 від обох частин рівняння: $y = 3$."
5,tutor_answer,"(7, 3)"
6,judge_success,True
7,judge_has_numbered_solution,True
8,judge_has_valid_answer_format,True
9,judge_has_valid_description_format,True


In [30]:
display(results)

bool_columns = ['tutor_success', 'judge_success', 'judge_has_numbered_solution', 'judge_has_valid_answer_format', 'judge_has_valid_description_format']
failures = results[~results[bool_columns].all(axis=1)]

print(f"Failures: {len(failures)} / {len(results)}")
if not failures.empty:
    display(failures)

,user_input,tutor_success,tutor_type,judge_success,judge_has_numbered_solution,judge_has_valid_answer_format,judge_has_valid_description_format
0,Арифметичні дії,True,Задача,True,True,True,True
1,Геометрія,True,Задача,True,True,True,True
2,Спрощення виразів,True,Задача,True,True,True,True
3,Нерівності,True,Задача,True,True,True,True
4,Системи рівнянь,True,Задача,True,True,True,True
5,Обчисліть значення виразу: $100 - (15 + 5) \times ...,True,Задача,True,True,True,True
6,Чому дорівнює сума внутрішніх кутів довільного три...,True,Тест,True,True,True,True
7,"Спростіть вираз, використовуючи формулу різниці кв...",True,Задача,True,True,True,True
8,Який проміжок є розв'язком нерівності $x > 5$? \n A...,True,Тест,True,True,True,True
9,Розв'яжіть систему рівнянь: $\begin{cases} x + y =...,True,Задача,True,True,True,True


Failures: 0 / 10
